# from package data

In [1]:
import numpy as np
import os

In [21]:
np.array([2.67662060e-05,2.59721181e-05,1.96718356e-05,1.22474419e-03])*6*3600

array([ 0.57815005,  0.56099775,  0.42491165, 26.4544745 ])

In [8]:
DATAROOT="/nvme/zhangtianning/datasets/ERA5/numpy_set_float32"
#Nabla_cdot_V_l    = np.load(os.path.join(DATAROOT,"Nabla_cdot_V_train.npy"  ))
year_data_l       = np.load(os.path.join(DATAROOT,"all_data_test.npy"))
Vphysics_dx_l     = np.load(os.path.join(DATAROOT,"Vphysics_dx_test.npy"))
Vphysics_dy_l     = np.load(os.path.join(DATAROOT,"Vphysics_dy_test.npy"))
Field_dx_l        = np.load(os.path.join(DATAROOT,"Field_dx_test.npy"))
Field_dy_l        = np.load(os.path.join(DATAROOT,"Field_dy_test.npy"))
# InteractionPart_l = np.load(os.path.join(DATAROOT,"InteractionPart_train.npy"))

In [12]:
year_data_l = year_data_l[...,1:-1,:]

In [13]:
u = year_data_l[:,0:1,]
v = year_data_l[:,1:2,]
T = year_data_l[:,2:3,]
p = year_data_l[:,3:4,]

In [15]:
compute_vphysics_dx_l=Field_dx_l*u + year_data_l*Field_dx_l[:,0:1]

In [51]:
InteractionPart_l=np.concatenate([InteractionPart_l[:,0:1,:,1:-1,:]/Hdx.reshape(1,1,1,32,1)[...,1:-1,:]*6*3600,
          InteractionPart_l[:,1:2,:,1:-1,:]/Hdy*6*3600
         ],1)

In [15]:
Hdx = 6371000*np.sin(np.linspace(0,720*np.pi/721,32))*2*np.pi/1440.0#m
Hdy = 6371000*np.pi/720.0

Vphysics_dx_l = Vphysics_dx_l[...,1:-1,:]/Hdx.reshape(1,1,1,32,1)[...,1:-1,:]*6*3600
Vphysics_dy_l = Vphysics_dy_l[...,1:-1,:]/Hdy*6*3600

Field_dx_l= Field_dx_l[...,1:-1,:]/Hdx.reshape(1,1,1,32,1)[...,1:-1,:]*6*3600
Field_dy_l= Field_dy_l[...,1:-1,:]/Hdy*6*3600

Field = year_data_l

Nabla_cdot_V   = Field_dx_l[:,0:1] + Field_dy_l[:,1:2]
Nabla_V_Field  = Nabla_cdot_V*Field[...,1:-1,:] #(Batch, 4, z, y ,x)

PhysicsPart    = -Vphysics_dx_l - Vphysics_dy_l  + Nabla_V_Field #(Batch,4,z, y ,x)

Delta= year_data_l[1:,:,:,1:-1,:]-year_data_l[:-1,:,:,1:-1,:]

In [119]:
Field.shape

(54056, 4, 3, 32, 64)

In [213]:
Omega = 2*np.pi/(24*3600)
omg   = 2*Omega*np.sin(np.linspace(0,720*np.pi/721,32)).reshape(1,1,32,1)[...,1:-1,:]

Fcols = np.stack([omg*Field[:,1,...,1:-1,:],
                 -omg*Field[:,0,...,1:-1,:]
                 ],
                 1)*(6*3600)

In [214]:
cent  = 6371000*np.sin(np.linspace(0,720*np.pi/721,32)).reshape(1,1,32,1)[...,1:-1,:]*(Omega**2)*np.sin(np.linspace(0,720*np.pi/721,32)).reshape(1,1,32,1)[...,1:-1,:]

In [96]:
Speed_PhysicsPart = Fcols + PhysicsPart[:,:2]

In [97]:
#record={}

In [113]:
names = [
"Vphysics_dx_l    ",
"Vphysics_dy_l    ",
"Field_dx_l       ",
"Field_dy_l       ",
"Delta            ",
"PhysicsPart      ",
"InteractionPart_l",
"Speed_PhysicsPart",
"Coriolis",
"Field"
]
datas= [
Vphysics_dx_l    ,
Vphysics_dy_l    ,
Field_dx_l       ,
Field_dy_l       ,
Delta,
PhysicsPart,
InteractionPart_l,
Speed_PhysicsPart,
Fcols,
Field
]

for name, data in zip(names,datas):
    if name not in record:
        record[name]={'mean':data.mean(axis=(0,2,3,4)),'std':data.std(axis=(0,2,3,4))}
        means = record[name]['mean']
        stds  = record[name]['std']
        print(f"""{name}-> mean:{means} 
                      -> std:{stds}""")

Field-> mean:[2.5879788e+00 7.1378373e-02 2.2813908e+04 2.6789578e+02] 
                      -> std:[9.3581161e+00 6.9625368e+00 2.2779877e+04 2.0678736e+01]


In [71]:
import pandas as pd

In [125]:
table={}
item_names = ['u','v','phi','T']
for name, val in record.items():
    table[name]={}
    means = record[name]['mean']
    stds  = record[name]['std']
    for item_name, val in zip(item_names,means):
        table[name][item_name+'.mean']=val
    for item_name, val in zip(item_names,stds):
        table[name][item_name+'.std']=val
df= pd.DataFrame(data=table).transpose()
df

,u.mean,v.mean,phi.mean,T.mean,u.std,v.std,phi.std,T.std
Vphysics_dx_l,-0.010389,0.010372,-15.440293,-1.567747,10.928425,8.855646,14466.454603,134.355430
Vphysics_dy_l,-0.014326,-0.038923,-3.949892,0.661720,8.248512,8.171262,14285.976562,126.136284
Field_dx_l,-0.006099,-0.002192,-0.229208,0.000843,0.507612,0.638157,23.932706,0.430413
Field_dy_l,-0.006519,0.002594,-1.749277,-0.019260,0.626688,0.472728,27.701265,0.405070
Delta,0.000021,-0.000002,-0.000546,0.000026,3.280018,3.796076,215.981201,1.982062
PhysicsPart,0.011243,0.056157,-6.752281,-0.027942,8.337613,8.863453,540.010947,3.695993
InteractionPart_l,0.000843,-0.019260,NaN,NaN,0.430413,0.405071,NaN,NaN
Speed_PhysicsPart,0.161378,-5.020204,NaN,NaN,16.318185,22.784317,NaN,NaN
Coriolis,0.150135,-5.076361,NaN,NaN,14.020784,20.969825,NaN,NaN
Field,2.587979,0.071378,22813.908203,267.895782,9.358116,6.962537,22779.876953,20.678736


In [55]:
absdelta = np.abs(PhysicsPart[:-1])-np.abs(Delta)

In [31]:
Delta.std(axis=(0,2,4))

array([[6.8582321e-04, 3.0365341e+00, 2.9900851e+00, 3.2115424e+00,
        3.3119178e+00, 3.6096151e+00, 3.9751532e+00, 4.0409827e+00,
        3.9987764e+00, 3.6142375e+00, 3.1261528e+00, 2.5146220e+00,
        2.0914984e+00, 1.8692865e+00, 1.7569691e+00, 1.5595763e+00,
        1.4677874e+00, 1.5765243e+00, 1.8257756e+00, 2.1604939e+00,
        2.6817858e+00, 3.2946739e+00, 3.9003646e+00, 4.4444013e+00,
        4.8586063e+00, 5.0699425e+00, 5.0095835e+00, 4.4081330e+00,
        3.1421418e+00, 2.3717465e+00, 2.1505504e+00, 1.7760234e+00],
       [6.7546038e-04, 3.0193648e+00, 3.1491852e+00, 3.3569860e+00,
        3.5339034e+00, 3.9186771e+00, 4.3958836e+00, 4.6796927e+00,
        4.6748643e+00, 4.2665548e+00, 3.4933813e+00, 2.6541188e+00,
        2.1811774e+00, 1.9074473e+00, 1.7990435e+00, 1.5918880e+00,
        1.5058992e+00, 1.6057800e+00, 1.8195912e+00, 2.2459192e+00,
        2.8685102e+00, 3.7508392e+00, 4.8192420e+00, 5.9064946e+00,
        6.6435027e+00, 6.7405386e+00, 5.9491496

In [84]:
name = "Vphysics_dx_l";data = Vphysics_dx_l
print(f"{name}-> mean:{data.mean():.3e} std:{data.std():.3e}")
name = "Vphysics_dy_l";data = Vphysics_dy_l
print(f"{name}-> mean:{data.mean():.3e} std:{data.std():.3e}")

Vphysics_dx_l-> mean:-4.346e+00 std:7.261e+03
Vphysics_dy_l-> mean:-8.354e-01 std:7.143e+03


# from origin raw data

In [92]:
# %load downsamplefrom720.py
import torch
import numpy as np
import xarray as xr
from concurrent.futures import ThreadPoolExecutor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# import petrel_client
# from petrel_client.client import Client
# client_config_file = "/mnt/lustre/share/pymc/mc.conf"


# surface_pressure inf, mean_sea_level_pressure inf, 50h_geopotential inf
vnames = [
    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature', 'surface_pressure', 'mean_sea_level_pressure',
    '1000h_u_component_of_wind', '1000h_v_component_of_wind', '1000h_geopotential',
    '850h_temperature', '850h_u_component_of_wind', '850h_v_component_of_wind', '850h_geopotential', '850h_relative_humidity',
    '500h_temperature', '500h_u_component_of_wind', '500h_v_component_of_wind', '500h_geopotential', '500h_relative_humidity',
    '50h_geopotential',
    'total_column_water_vapour',
    # '850h_geopotential',
    # 'surface_pressure', 'mean_sea_level_pressure', '50h_geopotential',
    # '850h_geopotential'
]
vnames_short = [
    'u10', 'v10', 't2m', 'sp', 'msl',
    'u', 'v', 'z',
    't', 'u', 'v', 'z', 'r',
    't', 'u', 'v', 'z', 'r',
    'z',
    'tcwv'
    # 'z',
    # 'sp', 'msl', 'z',
    # 'z',
]
physics_index = [5, 9, 14, #u
                 6, 10, 15,#v
                 2, 8, 13, #T
                 7, 11, 16,#phi
                ]

years = range(1979, 1980)
#years = range(1988, 1989)
Ashape = (721, 1440)
import os
import subprocess
w = 721
h = 1440
CACHEROOT = "/nvme/zhangtianning/datasets/ERA5/grid"
DATAROOT  = "/nvme/zhangtianning/datasets/ERA5/numpy"
def get_numpy_from_grid(vname_pair, year):
    #return np.random.randn(1500,w,h)
    vname, vname_short = vname_pair
    fname   = f'{CACHEROOT}/{year}/{vname}/{vname:s}-{year:d}.grib'
    if len(os.listdir(CACHEROOT))>0:
        for tyear in os.listdir(CACHEROOT):
            if os.path.exists(os.path.join(DATAROOT,f"all_data_{tyear}.npy")):
                pass
                #os.system(f"rm -r {CACHEROOT}/{tyear}")
                #os.system(f"rm -r {CACHEROOT}/{year}")
    if not os.path.exists(fname):
        print(f"download {fname}")
        os.system(f"aws s3 --endpoint-url=http://10.140.2.254:80 --profile chenzhuo cp s3://era5/{vname}/{vname}-{year}.grib {fname}")
    ds = xr.open_dataset(fname, engine="cfgrib")
    ds_array = ds.data_vars[vname_short]
    return ds_array

def Drive(x,position=-1,dim=3,mode='circular'):
    expandQ=False
    if len(x.shape) == dim + 1 and x.shape[1]!=1:
        x = x.unsqueeze(1)
        expandQ = True
    conv_engine = [torch.conv1d, torch.conv2d, torch.conv3d][dim-1]
    weight = torch.Tensor([1/12, -8/12, 0, 8/12, -1/12]).cuda()
    weight = weight[(None,)*(dim + 1)].transpose(position,-1)
    pad_num = 2
    padtuple = [0]*dim*2
    padtuple[(-position-1)*2]   = pad_num
    padtuple[(-position-1)*2+1] = pad_num
    padtuple = tuple(padtuple)
    x = conv_engine(F.pad(x, padtuple, mode=mode), weight)
    return x.squeeze(1) if expandQ else x


Dx = lambda x:Drive(x,position=-1,dim=3,mode='circular')
Dy = lambda x:Drive(x,position=-2,dim=3,mode='replicate')
Dz = lambda x:Drive(x,position=-3,dim=3,mode='replicate')
from tqdm import tqdm


x = np.linspace(0, 1440, 97)[:-1]
y = np.linspace(0, 720, 49)
x = x.astype('int')
y = y.astype('int')
x, y = np.meshgrid(x, y)

In [3]:
# means=[-0.000111572605,
#         5759.5347,
#         -5.081689e-06,
#         -0.17256598,
#         -2.5754157e-06,
#         -0.20135987,
#         2577.635,
#         0.00091999245]

In [4]:
# stds=[0.8410478,
#     15005.763,
#     5970.817,
#     10405.471,
#     8.406051,
#     465.392,
#     10321.853,
#     5.363033]

In [93]:
from tqdm.notebook import tqdm

In [94]:
def pad_data(year_data):
    nan_count= np.isnan(year_data).sum()
    nanlevel = 0
    if nan_count > 0:
        t, p, xx, yy = np.where(np.isnan(year_data))
        year_data[p,xx,yy] = (year_data[t,p,xx-1,yy] + year_data[t,p,xx+1,yy] 
                            + year_data[t,p,xx,yy-1] + year_data[t,p,xx,yy+1])/4
    
        
        if np.isnan(year_data).sum() > 0:
            print("bad nan detect")
            nanlevel=1 
            t,p, xx, yy = np.where(np.isnan(year_data))
            trail_x = xx
            while np.isnan(year_data[t,p,trail_x,yy]).any():
                trail_x+=1
            year_data[t,p,xx,yy] = year_data[t,p,trail_x,yy]
    return year_data, nan_count, nanlevel

In [157]:
for year in years:    
    print(f"loading {year} data ................", end="")
    year_datas= []
    for physics_name_idx in physics_index:
        vname = vnames[physics_name_idx]
        vname_short= vnames_short[physics_name_idx]
        year_datas.append(get_numpy_from_grid([vname, vname_short], year))
    #year_datas= np.stack(year_datas,1)
    print("done!")
    print(f"postprecessing ...............", end="")
    Vphysics_dx_l    =[]
    Vphysics_dy_l    =[]
    Field_dx_l       =[]
    Field_dy_l       =[]
    Field_dz_l       =[]
    InteractionPart_l=[]
    year_data_l      =[]
    Nabla_cdot_V_l   =[]
    #year_datas= torch.Tensor(year_datas.reshape(len(year_datas),4,3,w,h))
    i=0
    nanlist= []
    for year_data in tqdm(zip(*year_datas)):
        i+=1
        year_data= np.stack(year_data,0)
        #break
        nan_count= np.isnan(year_data).sum()
        if nan_count > 0:
            p, xx, yy = np.where(np.isnan(year_data))
            year_data[p,xx,yy] = (year_data[p,xx-1,yy] + year_data[p,xx+1,yy] 
                                + year_data[p,xx,yy-1] + year_data[p,xx,yy+1])/4
        nanlevel=0
        if np.isnan(year_data).sum() > 0:
            print("bad nan detect")
            nanlevel=1 
            p, xx, yy = np.where(np.isnan(year_data))
            trail_x = xx
            while np.isnan(year_data[p,trail_x,y]).any():
                trail_x+=1
            year_data[p,xx,yy] = year_data[p,trail_x,yy]
        nanlist.append([np.isnan(year_data).sum(),nanlevel])
        
        year_data= torch.Tensor(year_data).unsqueeze(0)
        
        year_data= year_data.reshape(1,4,3,w,h).cuda()
        u = year_data[:, 0:1]  # (Batch, 1, z, y ,x)
        v = year_data[:, 1:2]  # (Batch, 1, z, y ,x)
        T = year_data[:, 2:3]  # (Batch, 1, z, y ,x)
        p = year_data[:, 3:4]  # (Batch, 1, z, y ,x)
        V = torch.cat([u, v], 1)  # (Batch, 3, z, y ,x)
        Nabla_cdot_V    = (Dx(u[:, 0]) + Dy(v[:, 0])).unsqueeze(1)  # (Batch, 1, z, y ,x)
        Vphysics        = torch.stack([V*u, V*v, V*T, V*p], 1)
        Vphysics_dx     = Dx(Vphysics[:, :, 0].flatten(0, 1)).reshape(year_data.shape)  # (Batch, 4, z, y ,x)
        Vphysics_dy     = Dy(Vphysics[:, :, 1].flatten(0, 1)).reshape(year_data.shape)  # (Batch, 4, z, y ,x)
        Field_dx        = Dx(year_data.flatten(0,1)).reshape(year_data.shape)#(Batch, 4, z, y ,x)
        Field_dy        = Dy(year_data.flatten(0,1)).reshape(year_data.shape)#(Batch, 4, z, y ,x)
        Field_dz        = Dz(year_data.flatten(0,1)).reshape(year_data.shape)#(Batch, 4, z, y ,x)
        InteractionPart = torch.stack([Dx(p[:, 0]),Dy(p[:, 0])], 1)  # (Batch,2, z, y ,x)
        
        Nabla_cdot_V_l.append(   (   Nabla_cdot_V[...,y,x].cpu().numpy()))
        year_data_l.append(      (      year_data[...,y,x].cpu().numpy()))
        Vphysics_dx_l.append(    (    Vphysics_dx[...,y,x].cpu().numpy()))
        Vphysics_dy_l.append(    (    Vphysics_dy[...,y,x].cpu().numpy()))
        Field_dx_l.append(       (       Field_dx[...,y,x].cpu().numpy()))
        Field_dy_l.append(       (       Field_dy[...,y,x].cpu().numpy()))
        Field_dz_l.append(       (       Field_dz[...,y,x].cpu().numpy()))
        InteractionPart_l.append((InteractionPart[...,y,x].cpu().numpy()))        

loading 1979 data ................done!
postprecessing ...............

0it [00:00, ?it/s]

In [158]:
Nabla_cdot_V    = np.concatenate(Nabla_cdot_V_l   )[...,1:-1,:]
year_data       = np.concatenate(year_data_l      )[...,1:-1,:]
Vphysics_dx     = np.concatenate(Vphysics_dx_l    )[...,1:-1,:]
Vphysics_dy     = np.concatenate(Vphysics_dy_l    )[...,1:-1,:]
Field_dx        = np.concatenate(Field_dx_l       )[...,1:-1,:]
Field_dy        = np.concatenate(Field_dy_l       )[...,1:-1,:]
Field_dz        = np.concatenate(Field_dz_l       )[...,1:-1,:]
InteractionPart = np.concatenate(InteractionPart_l)[...,1:-1,:]

In [160]:
print((Nabla_cdot_V   ).mean(),end=','),print((Nabla_cdot_V   ).std())
print((year_data      ).mean(),end=','),print((year_data      ).std())
print((Vphysics_dx    ).mean(),end=','),print((Vphysics_dx    ).std())
print((Vphysics_dy    ).mean(),end=','),print((Vphysics_dy    ).std())
print((Field_dx       ).mean(),end=','),print((Field_dx       ).std())
print((Field_dy       ).mean(),end=','),print((Field_dy       ).std())
print((Field_dz       ).mean(),end=','),print((Field_dz       ).std())
print((InteractionPart).mean(),end=','),print((InteractionPart).std())

0.0081150625,0.8149286
5785.038,15093.026
-1.3076661,5933.7363
15.531034,9137.067
-0.043823842,8.15883
-0.6715402,18.189743
4820.2764,10079.634
-1.4213241,28.157673


(None, None)

In [166]:
Delta= (year_data[1:]-year_data[:-1])/(6*3600)

Hdx = 6371000*np.sin(np.linspace(0,720,49)/720*np.pi)*2*np.pi/1440.0
Hdx = Hdx.reshape(1,1,1,49,1)[...,1:-1,:]
Hdy = 6371000*np.pi/720.0

Vphysics_dx    = Vphysics_dx/Hdx
Vphysics_dy    = Vphysics_dy/Hdy
Field_dx       = Field_dx/Hdx
Field_dy       = Field_dy/Hdy
Field          = year_data
Nabla_cdot_V   = Field_dx[:,0:1] + Field_dy[:,1:2]
Nabla_V_Field  = Nabla_cdot_V*Field #(Batch, 4, z, y ,x)

In [164]:
InteractionPart=np.concatenate([
    InteractionPart[:,0:1]/Hdx,
    InteractionPart[:,1:2]/Hdy
 ],1)

In [165]:
PhysicsPart    = -Vphysics_dx - Vphysics_dy  + Nabla_V_Field #(Batch,4,z, y ,x)

Pure_Physics = PhysicsPart[:,:2] - InteractionPart

Residual_Part =  Delta[:,:2] - Pure_Physics[:-1]

Residual_Part2=  Delta[:,:2] - PhysicsPart[:-1][:,:2]

In [167]:
record={}

names = [
    "Vphysics_dx",
    "Vphysics_dy",
    "Field_dx",
    "Field_dy",
    "Delta",
    "PhysicsPart",
    "InteractionPart",
    "Field",
    "Pure_Physics",
    "Residual_Part",
    "Residual_Part2",
    "Nabla_V_Field",
    "Nabla_cdot_V"
]
datas= [
    Vphysics_dx    ,
    Vphysics_dy    ,
    Field_dx       ,
    Field_dy       ,
    Delta,
    PhysicsPart,
    InteractionPart,
    Field,
    Pure_Physics,
    Residual_Part,
        Residual_Part2,
        Nabla_V_Field,
        Nabla_cdot_V
]

for name, data in zip(names,datas):
    if name not in record:
        record[name]={'mean':data.mean(axis=(0,2,3,4)),'std':data.std(axis=(0,2,3,4))}
        means = record[name]['mean']
        stds  = record[name]['std']
        print(f"""{name}-> mean:{means} 
                      -> std:{stds}""")

In [189]:
96/2/2/2/2

6.0

In [ ]:
import pandas as pd
def show_record(record):
    table={}
    item_names = ['u','v','T','phi']
    for name, val in record.items():
        table[name]={}
        means = record[name]['mean']
        stds  = record[name]['std']
        scale = 1 if name in ['Field'] else 6*3600
        for item_name, val in zip(item_names,means):
            table[name][item_name+'.mean']=val*scale
        for item_name, val in zip(item_names,stds):
            table[name][item_name+'.std']=val*scale
    df= pd.DataFrame(data=table).transpose()
    return df

In [199]:
show_record(record)

,u.mean,v.mean,T.mean,phi.mean,u.std,v.std,T.std,phi.std
Vphysics_dx,-0.018282,0.007609,-0.334980,-9.062807,10.737418,8.737609,136.285303,14521.652115
Vphysics_dy,-0.026208,-0.022910,1.613607,46.706918,8.160330,7.995125,127.352970,14198.753643
Field_dx,-0.001391,-0.001975,0.003829,-0.156239,0.518175,0.629573,0.440272,24.520506
Field_dy,-0.005270,0.006639,-0.017943,-2.070606,0.633020,0.479048,0.416807,28.196510
Delta,0.000148,-0.000085,-0.000161,-0.006712,3.252417,3.739944,1.961468,210.937078
PhysicsPart,0.027327,0.056185,-0.024790,-6.584695,8.201135,8.708302,3.605344,533.826262
InteractionPart,-0.156239,-2.070608,NaN,NaN,24.520506,28.196510,NaN,NaN
Field,2.791147,0.084584,268.888611,22868.474609,9.514891,7.041424,20.263653,22848.212891
Pure_Physics,0.183566,2.126793,NaN,NaN,26.215173,29.649409,NaN,NaN
Residual_Part,-0.183517,-2.127055,NaN,NaN,26.441180,29.582388,NaN,NaN


# 检查多step时序是否满足简单约束

In [196]:
print(DeltaPhysicsPart.mean(axis=(0,2,3,4)))

[-2.52965258e-09 -7.43957267e-09  8.72048350e-10 -2.59878779e-07]


In [197]:
print(DeltaPhysicsPart.std(axis=(0,2,3,4)))

[0.00043707 0.00048891 0.00015177 0.01749836]


In [220]:
record2={}

In [225]:
Pure_Physics2 = PhysicsPart[:,:2] + InteractionPart

In [227]:
names = [
"Delta_Field",
"Delta_Delta",
"Delta_Vphysics_dx",
"Delta_Vphysics_dy",
"Delta_Field_dx",
"Delta_Field_dy",
"Delta_PhysicsPart",
"Delta_InteractionPart",
"Delta_Nabla_V_Field",
"Delta_Nabla_cdot_V",
"Delta_Pure_Physics",
"Delta_Pure_Physics2",
]
datas= [
Field[1:]-Field[:-1],
Delta[1:]-Delta[:-1],
    
Vphysics_dx[1:]-Vphysics_dx[:-1],
Vphysics_dy[1:]-Vphysics_dy[:-1],
Field_dx[1:]-Field_dx[:-1],
Field_dy[1:]-Field_dy[:-1],
    
PhysicsPart[1:]-PhysicsPart[:-1],
InteractionPart[1:]-InteractionPart[:-1],

Nabla_V_Field[1:]-Nabla_V_Field[:-1],
Nabla_cdot_V[1:]-Nabla_cdot_V[:-1],
Pure_Physics[1:]-Pure_Physics[:-1],
Pure_Physics2[1:]-Pure_Physics2[:-1]
     
]

for name, data in zip(names,datas):
    if name not in record2:
        record2[name]={'mean':data.mean(axis=(0,2,3,4)),'std':data.std(axis=(0,2,3,4))}
        means = record2[name]['mean']
        stds  = record2[name]['std']
        print(f"""{name}-> mean:{means} 
                      -> std:{stds}""")

Delta_Pure_Physics2-> mean:[-1.33596632e-09 -6.55722599e-08] 
                      -> std:[0.00073334 0.00071037]


In [228]:
show_record(record2)

,u.mean,v.mean,T.mean,phi.mean,u.std,v.std,T.std,phi.std
Delta_Field,3.205036,-1.838042e+00,-3.472811e+00,-144.973515,70252.250290,80782.826042,42367.703247,4.556240e+06
Delta_Delta,0.000116,2.404016e-05,-1.336383e-04,0.002357,4.457346,5.089614,2.716328,2.433105e+02
Delta_Vphysics_dx,-0.000002,-2.570180e-05,3.057546e-04,-0.007261,12.448258,10.505241,135.833837,1.640126e+04
Delta_Vphysics_dy,0.000029,1.059230e-04,-1.257631e-03,-0.050401,9.509203,9.302144,134.664940,1.664235e+04
Delta_Field_dx,0.000001,-5.571974e-07,2.135792e-07,0.000026,0.504207,0.645084,0.308346,1.331160e+01
Delta_Field_dy,0.000001,-4.624348e-06,-1.613735e-05,-0.001256,0.606013,0.498960,0.279796,1.172719e+01
Delta_PhysicsPart,-0.000055,-1.606948e-04,1.883624e-05,-0.005613,9.440750,10.560448,3.278242,3.779647e+02
Delta_InteractionPart,0.000026,-1.255666e-03,NaN,NaN,13.311601,11.727190,NaN,NaN
Delta_Nabla_V_Field,-0.000028,-8.047395e-05,-9.330440e-04,-0.063275,11.141464,8.473336,215.393648,2.783000e+04
Delta_Nabla_cdot_V,-0.000003,NaN,NaN,NaN,0.808611,NaN,NaN,NaN
